In [346]:
##############建立时间索引字典##############

from datetime import timedelta
from datetime import datetime
def gen_dates(b_date, count):
    hour = timedelta(hours=1)
    for i in range(count+1):
        yield b_date + hour*i


def get_date_list(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.strptime("2000-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
    if end is None:
        end = datetime.now()
    data = []
    for d in gen_dates(start, int((end-start).total_seconds()/3600)):
        data.append(str(d))
    return data

Start = datetime.strptime("2017-02-01 00:00:00", "%Y-%m-%d %H:%M:%S")
End = datetime.strptime("2017-05-18 08:00:00", "%Y-%m-%d %H:%M:%S")
train_list1 = get_date_list(Start , End)

Start = datetime.strptime("2017-07-08 16:00:00", "%Y-%m-%d %H:%M:%S")
End = datetime.strptime("2018-01-31 15:00:00", "%Y-%m-%d %H:%M:%S")
train_list2 = get_date_list(Start , End)

train_list = train_list1 + train_list2

Start = datetime.strptime("2017-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
End = datetime.strptime("2017-07-01 15:00:00", "%Y-%m-%d %H:%M:%S")
test_list = get_date_list(Start , End)

for i in train_list:
    train_dict[i] = []
for i in test_list:
    test_dict[i] = []

In [347]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

############beijing 17-18的污染物csv读取################
beijing_17_18_aq = pd.read_csv('/home/liyouru/.ipython/profile_myserver/dataset/KDD_Cup2018/beijing_17_18_aq_full.csv')

# ########建立时间字典######### 2017-02-01 00:00:00 -----2017-05-18 8:00:00

# train_dict = {}   

# for i in range(2017,2018):
#     year = str(i)
#     for j in range(1,6):
#         month = '0' + str(j)
#         if i == 2017 and j == 1:
#             continue
#         for k in range(1,32):
#             if i == 2017 and j == 5 and k ==19:
#                 break
#             if k < 10:
#                 day = '0' + str(k)
#             if k >= 10:
#                 day = str(k)
#             if j == 2 and k>28:
#                 break
#             if j == 4 or j == 6 or j == 9 or j == 11:
#                 if k>30:
#                     break

#             for t in range(0,24):
#                 if i == 2017 and j == 5 and k ==18 and t >= 9:
#                     break
#                 hour = str(t)
#                 if t < 10:
#                     train_dict[year+'-'+month+'-'+day+' 0'+hour+':00:00'] = []
#                 if t >= 10:
#                     train_dict[year+'-'+month+'-'+day+' '+hour+':00:00'] = []
                    
# ########建立时间字典######### 2017-07-08 16:00:00 -----2018-01-31 15:00:00

# for i in range(2017,2019):
#     year = str(i)
#     for j in range(1,13):
#         if j < 10:
#             month = '0' + str(j)
#         if j >= 10:
#             month = str(j)
#         if i == 2017 and j == 1:
#             continue
#         if i == 2018 and j > 1:
#             break
#         for k in range(1,32):
#             if i == 2017 and j < 7:
#                 break
#             elif j == 7 and k < 8:
#                 continue
#             if k < 10:
#                 day = '0' + str(k)
#             if k >= 10:
#                 day = str(k)
#             if j == 2 and k>28:
#                 break
#             if j == 4 or j == 6 or j == 9 or j == 11:
#                 if k>30:
#                     break

#             for t in range(0,24):
#                 if i == 2017 and j == 7 and k == 8 and t < 16:
#                     continue
#                 if i == 2018 and j == 1 and k ==31 and t == 16:
#                     break
#                 hour = str(t)
#                 if t < 10:
#                     train_dict[year+'-'+month+'-'+day+' 0'+hour+':00:00'] = []
#                 if t >= 10:
#                     train_dict[year+'-'+month+'-'+day+' '+hour+':00:00'] = []
                    
# ########建立时间字典######### 2017-06-01 00:00:00 -----2017-07-01 15:00:00        
# test_dict = {}   

# for i in range(2017,2018):
#     year = str(i)
#     for j in range(6,8):
#         month = '0' + str(j)
#         for k in range(1,32):
#             if i == 2017 and j == 7 and k == 2:
#                 break
#             if k < 10:
#                 day = '0' + str(k)
#             if k >= 10:
#                 day = str(k)
#             if j == 2 and k>28:
#                 break
#             if j == 4 or j == 6 or j == 9 or j == 11:
#                 if k>30:
#                     break

#             for t in range(0,24):
#                 if i == 2017 and j == 7 and k == 1 and t >= 16:
#                     break
#                 hour = str(t)
#                 if t < 10:
#                     test_dict[year+'-'+month+'-'+day+' 0'+hour+':00:00'] = []
#                 if t >= 10:
#                     test_dict[year+'-'+month+'-'+day+' '+hour+':00:00'] = []
#                 #########建立时间字典完毕############


count = 0
index = []
time_win_PM25 = []
time_win_PM10 = []
time_win_CO   = []

for i in tqdm(beijing_17_18_aq['utc_time']):
    
#     if i.split('-')[0] + i.split('-')[1] != '201701':            ###取48小时时序窗口####
#         if i.split(' ')[0] == '2017-05-31' or i.split(' ')[0] == '2017-07-01' or i.split('-')[1] == '06':
#             test_dict[i].extend(beijing_17_18_aq['PM2.5'][count-48:count])
#             test_dict[i].extend(beijing_17_18_aq['PM10'][count-48:count])
#             test_dict[i].extend(beijing_17_18_aq['CO'][count-48:count])
        
#         else:
#             train_dict[i].extend(beijing_17_18_aq['PM2.5'][count-48:count])
#             train_dict[i].extend(beijing_17_18_aq['PM10'][count-48:count])
#             train_dict[i].extend(beijing_17_18_aq['CO'][count-48:count])
        
    if i.split('-')[1] == '01' and i.split('-')[0] == '2017':
         index.append(count)
    count += 1
    
beijing_17_18_aq.drop(index , inplace= True) 



#####处理重复的时间点###########
beijing_17_18_aq.drop(columns='Unnamed: 0' ,axis=1 ,inplace=True)
beijing_17_18_aq.drop_duplicates(inplace=True)  ##删除重行
beijing_17_18_aq.reset_index(drop=True, inplace=True)
# count = 0
# hour_last = 0
# date_last = '2017-01-31 00:00:00'

# for i in beijing_17_18_aq['utc_time']:
    
# #     if i == date_last:
# #         beijing_17_18_aq.drop(count, inplace=True)
#     if int(i.split(' ')[1].split(':')[0]) - int(i.split(' ')[1].split(':')[0]) != 1 and int(i.split(' ')[1].split(':')[0]) - int(i.split(' ')[1].split(':')[0]) != -23:
#         beijing_17_18_aq.insert(count , count+1)
#####处理重复  和   缺失的时间点###########

beijing_17_18_aq['lon'] = 0 ##经度
beijing_17_18_aq['lat'] = 0  ##纬度
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='dongsi_aq',['lon','lat']] = [116.417,39.929]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='tiantan_aq',['lon','lat']] = [116.407,39.886]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='guanyuan_aq',['lon','lat']] = [116.339,39.929]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='wanshouxigong_aq',['lon','lat']] = [116.352,39.878]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='aotizhongxin_aq',['lon','lat']] = [116.397,39.982]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='nongzhanguan_aq',['lon','lat']] = [116.461,39.937]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='wanliu_aq',['lon','lat']] = [116.287,39.987]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='beibuxinqu_aq',['lon','lat']] = [116.174,40.09]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='zhiwuyuan_aq',['lon','lat']] = [116.207,40.002]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='fengtaihuayuan_aq',['lon','lat']] = [116.279,39.863]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yungang_aq',['lon','lat']] = [116.146,39.824]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='gucheng_aq',['lon','lat']] = [116.184,39.914]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='fangshan_aq',['lon','lat']] = [116.136,39.742]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='daxing_aq',['lon','lat']] = [116.404,39.718]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yizhuang_aq',['lon','lat']] = [116.506,39.795]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='tongzhou_aq',['lon','lat']] = [116.663,39.886]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='shunyi_aq',['lon','lat']] = [116.655,40.127]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='pingchang_aq',['lon','lat']] = [116.23,40.217]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='mentougou_aq',['lon','lat']] = [116.106,39.937]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='pinggu_aq',['lon','lat']] = [117.1,40.143]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='hubeijing_17_18_aqou_aq',['lon','lat']] = [116.628,40.328]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='miyun_aq',['lon','lat']] = [116.832,40.37]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yanqin_aq',['lon','lat']] = [115.972,40.453]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='dingling_aq',['lon','lat']] = [116.22,40.292]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='badaling_aq',['lon','lat']] = [115.988,40.365]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='miyunshuiku_aq',['lon','lat']] = [116.911,40.499]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='donggaocun_aq',['lon','lat']] = [117.12,40.1]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yongledian_aq',['lon','lat']] = [116.783,39.712]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yufa_aq',['lon','lat']] = [116.3,39.52]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='liulihe_aq',['lon','lat']] = [116,39.58]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='qianmen_aq',['lon','lat']] = [116.395,39.899]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='yongdingmennei_aq',['lon','lat']] = [116.394,39.876]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='xizhimenbei_aq',['lon','lat']] = [116.349,39.954]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='nansanhuan_aq',['lon','lat']] = [116.368,39.856]
beijing_17_18_aq.loc[beijing_17_18_aq['stationId']=='dongsihuan_aq',['lon','lat']] = [116.483,39.939]




count = 0
Idx = 0  ####索引记录####
hour_last = 0
date_last = '2017-01-31 23:00:00'
beijing_17_18_aq_list = np.array(beijing_17_18_aq[['PM2.5','PM10','NO2','CO','O3','SO2','lon','lat']]).tolist()
for i in tqdm(np.array(beijing_17_18_aq['utc_time']).tolist()):
    if i.split(' ')[0] == '2017-07-01' or i.split('-')[1] == '06': ##测试集字典
        if i == '2017-06-01 00:00:00':
            date_last = '2017-05-31 23:00:00'
        diff = int(i.split(' ')[1].split(':')[0]) - int(date_last.split(' ')[1].split(':')[0]) #####时间毛差######
        if diff != 1 and diff != -23 and diff != -15:

            if diff > 0:
                count = diff - 1  ####缺几个时间点

                for j in range(count+1):
                    Hour_str = int(date_last.split(' ')[1].split(':')[0])+j+1
                    if  Hour_str < 10:
                        alt = i.split(' ')[0] + ' 0'+ str(Hour_str)+':00:00'
                    if Hour_str >= 10:
                        alt = i.split(' ')[0] + ' '+ str(Hour_str)+':00:00'
                    test_dict[alt].extend(beijing_17_18_aq_list[Idx])
            if diff < 0:
                count = diff + 23  ####缺几个时间点

                for j in range(count+1):
                    Hour_str = int(date_last.split(' ')[1].split(':')[0])+j+1
                    if Hour_str < 10:
                        alt = date_last.split(' ')[0] + ' 0'+ str(Hour_str)+':00:00'
                    elif Hour_str >= 10 and Hour_str < 24:
                        alt = date_last.split(' ')[0] + ' '+ str(Hour_str)+':00:00'
                    elif Hour_str >= 24 and Hour_str < 34:
                        alt = i.split(' ')[0] + ' 0'+ str(Hour_str-24)+':00:00'
                    elif Hour_str >= 34:
                        alt = i.split(' ')[0] + ' '+ str(Hour_str-24)+':00:00'
                    test_dict[alt].extend(beijing_17_18_aq_list[Idx])
        else :
            test_dict[i].extend(beijing_17_18_aq_list[Idx])
        date_last = i
    elif i.split(' ')[0] != '2017-05-28' and i.split(' ')[0] != '2017-05-29'  and i.split(' ')[0] != '2017-05-31':  ##训练集字典
        if i == '2017-07-08 16:00:00':
            date_last = '2017-07-08 15:00:00'
        diff = int(i.split(' ')[1].split(':')[0]) - int(date_last.split(' ')[1].split(':')[0]) #####时间毛差######
        if diff != 1 and diff != -23 and diff != -15:

            if diff > 0:
                count = diff - 1  ####缺几个时间点

                for j in range(count+1):
                    Hour_str = int(date_last.split(' ')[1].split(':')[0])+j+1
                    if  Hour_str < 10:
                        alt = i.split(' ')[0] + ' 0'+ str(Hour_str)+':00:00'
                    if Hour_str >= 10:
                        alt = i.split(' ')[0] + ' '+ str(Hour_str)+':00:00'
                    train_dict[alt].extend(beijing_17_18_aq_list[Idx])
            if diff < 0:
                count = diff + 23  ####缺几个时间点

                for j in range(count+1):
                    Hour_str = int(date_last.split(' ')[1].split(':')[0])+j+1
                    if Hour_str < 10:
                        alt = date_last.split(' ')[0] + ' 0'+ str(Hour_str)+':00:00'
                    elif Hour_str >= 10 and Hour_str < 24:
                        alt = date_last.split(' ')[0] + ' '+ str(Hour_str)+':00:00'
                    elif Hour_str >= 24 and Hour_str < 34:
                        alt = i.split(' ')[0] + ' 0'+ str(Hour_str-24)+':00:00'
                    elif Hour_str >= 34:
                        alt = i.split(' ')[0] + ' '+ str(Hour_str-24)+':00:00'
                    train_dict[alt].extend(beijing_17_18_aq_list[Idx])
        else :
            train_dict[i].extend(beijing_17_18_aq_list[Idx])
        date_last = i
  
    Idx += 1
    

# ##########处理缺失的时间点##########

beijing_17_18_aq.head()

100%|██████████| 280235/280235 [00:01<00:00, 269849.94it/s]


,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2,lon,lat
0,aotizhongxin_aq,2017-02-01 00:00:00,7.0,11.0,25.0,0.5,58.0,12.0,116.397,39.982
1,aotizhongxin_aq,2017-02-01 01:00:00,9.0,14.0,22.0,0.4,64.0,11.0,116.397,39.982
2,aotizhongxin_aq,2017-02-01 02:00:00,10.0,12.0,11.0,0.3,79.0,6.0,116.397,39.982
3,aotizhongxin_aq,2017-02-01 03:00:00,6.0,8.0,9.0,0.3,83.0,4.0,116.397,39.982
4,aotizhongxin_aq,2017-02-01 04:00:00,9.0,12.0,11.0,0.4,84.0,6.0,116.397,39.982


In [360]:
length = []
for k,v in train_dict.items():
    length.append(len(v))
    if len(v) > 4981:
        print(k)
        print(len(v))
print(max(length))
print(min(length))

2017-08-22 15:00:00
18526
2017-10-29 19:00:00
18526
2017-08-05 09:00:00
18526
2017-07-29 15:00:00
18526
2017-03-27 06:00:00
18526
2017-10-19 17:00:00
18526
2018-01-14 22:00:00
18526
2017-03-14 14:00:00
18526
2017-03-27 23:00:00
18526
2017-12-05 18:00:00
18526
2017-04-11 10:00:00
18526
2017-07-11 05:00:00
18526
2017-07-24 16:00:00
18526
2017-09-22 08:00:00
18526
2017-08-04 07:00:00
18526
2017-09-08 08:00:00
18526
2017-02-22 00:00:00
18526
2017-10-14 06:00:00
18526
2017-07-17 10:00:00
18526
2017-09-26 13:00:00
18526
2017-09-18 14:00:00
18526
2017-12-24 06:00:00
18526
2017-10-22 21:00:00
18526
2017-02-08 22:00:00
18526
2017-10-02 07:00:00
18526
2017-04-01 16:00:00
18526
2017-10-22 15:00:00
18526
2017-05-03 20:00:00
18526
2017-04-02 22:00:00
18526
2017-04-16 16:00:00
18526
2017-04-07 07:00:00
18526
2018-01-13 19:00:00
18526
2017-09-13 13:00:00
18526
2017-10-15 14:00:00
18526
2017-04-06 08:00:00
18526
2017-04-16 18:00:00
18526
2017-03-27 01:00:00
18526
2017-10-04 05:00:00
18526
2017-02-10 1

18526
2017-10-01 13:00:00
18526
2017-08-03 17:00:00
18526
2017-12-02 10:00:00
18526
2017-09-30 07:00:00
18526
2018-01-19 07:00:00
18526
2017-08-24 15:00:00
18526
2017-12-20 11:00:00
18526
2017-03-21 09:00:00
18526
2017-04-28 10:00:00
18526
2017-02-16 12:00:00
18526
2017-05-13 23:00:00
18526
2017-11-15 03:00:00
18526
2017-09-12 17:00:00
18526
2017-11-07 03:00:00
18526
2017-07-21 02:00:00
18526
2017-03-14 04:00:00
18526
2017-11-24 09:00:00
18526
2017-12-29 19:00:00
18526
2017-12-18 12:00:00
18526
2018-01-01 23:00:00
18526
2017-03-15 11:00:00
18526
2017-09-27 17:00:00
18526
2017-10-26 23:00:00
18526
2017-07-12 23:00:00
18526
2017-08-14 01:00:00
18526
2017-11-26 21:00:00
18526
2017-08-18 23:00:00
18526
2017-08-30 12:00:00
18526
2017-07-19 07:00:00
18526
2017-12-26 09:00:00
18526
2017-05-06 12:00:00
18526
2017-03-28 12:00:00
18526
2017-03-17 12:00:00
18526
2018-01-20 01:00:00
18526
2017-05-04 13:00:00
18526
2017-02-14 13:00:00
18526
2017-02-20 15:00:00
18526
2017-12-29 06:00:00
18526
2018-0

18526
2017-11-02 04:00:00
18526
2017-08-04 20:00:00
18526
2017-12-02 01:00:00
18526
2017-03-13 08:00:00
18526
2017-09-18 00:00:00
18526
2017-02-17 18:00:00
18526
2017-03-01 21:00:00
18526
2017-02-22 16:00:00
18526
2017-03-30 20:00:00
18526
2017-07-29 05:00:00
18526
2018-01-18 09:00:00
18526
2017-09-06 15:00:00
18526
2017-08-23 16:00:00
18526
2017-10-27 17:00:00
18526
2017-03-14 18:00:00
18526
2017-04-03 00:00:00
18526
2017-09-18 05:00:00
18526
2017-12-14 00:00:00
18526
2017-05-03 17:00:00
18526
2017-10-13 20:00:00
18526
2017-09-19 23:00:00
18526
2017-08-29 14:00:00
18526
2017-11-01 10:00:00
18526
2018-01-16 20:00:00
18526
2017-10-20 09:00:00
18526
2017-02-06 12:00:00
18526
2017-07-11 01:00:00
18526
2017-07-13 02:00:00
18526
2017-10-25 08:00:00
18526
2017-09-11 11:00:00
18526
2017-09-25 21:00:00
18526
2017-10-20 01:00:00
18526
2017-07-23 05:00:00
18526
2017-10-30 04:00:00
18526
2017-12-12 13:00:00
18526
2017-09-23 08:00:00
18526
2017-08-16 00:00:00
18526
2017-05-01 07:00:00
18526
2017-0

18526
2017-04-24 01:00:00
18526
2017-10-03 13:00:00
18526
2017-03-10 23:00:00
18526
2017-10-30 03:00:00
18526
2017-12-20 16:00:00
18526
2017-11-26 22:00:00
18526
2017-03-25 03:00:00
18526
2017-05-06 03:00:00
18526
2017-10-06 11:00:00
18526
2017-07-14 07:00:00
18526
2017-04-03 16:00:00
18526
2017-07-13 06:00:00
18526
2017-12-26 10:00:00
18526
2017-02-21 11:00:00
18526
2017-12-29 20:00:00
18526
2017-07-08 23:00:00
18526
2017-08-21 10:00:00
18526
2017-08-25 02:00:00
18526
2017-08-21 00:00:00
18526
2017-11-17 10:00:00
18526
2018-01-14 14:00:00
18526
2017-04-13 19:00:00
18526
2017-04-11 23:00:00
18526
2017-08-29 10:00:00
18526
2018-01-15 15:00:00
18526
2017-04-22 20:00:00
18526
2017-12-17 11:00:00
18526
2017-10-02 11:00:00
18526
2017-09-30 18:00:00
18526
2017-05-07 19:00:00
18526
2017-07-26 18:00:00
18526
2017-08-18 20:00:00
18526
2017-10-08 09:00:00
18526
2017-04-09 03:00:00
18526
2017-09-22 00:00:00
18526
2017-12-14 10:00:00
18526
2017-05-13 09:00:00
18526
2017-09-20 05:00:00
18526
2017-0

In [348]:
#############beijing 17-18 真实点 气象数据 #############
beijing_17_18_meo = pd.read_csv('/home/liyouru/.ipython/profile_myserver/dataset/KDD_Cup2018/beijing_17_18_meo_full.csv')

count = 0
index = []

for i in tqdm(beijing_17_18_meo['utc_time']):
    if i.split('-')[1] == '01' and i.split('-')[0] == '2017':
        index.append(count)
    count += 1
    
beijing_17_18_meo.drop(index , inplace= True)
beijing_17_18_meo.drop_duplicates(inplace=True)        ##删除重行
beijing_17_18_meo.reset_index(drop=True, inplace=True)###重置索引
#############wether量化###############
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Sunny/clear' , 'weather'] = 0
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Snow' , 'weather'] = 1
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Sleet' , 'weather'] = 2
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Sand' , 'weather'] = 3
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Rain/Snow with Hail' , 'weather'] = 4
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Rain with Hail' , 'weather'] = 5
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Rain' , 'weather'] = 6
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Haze' , 'weather'] = 7
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Fog' , 'weather'] = 8
beijing_17_18_meo.loc[beijing_17_18_meo['weather'] == 'Dust' , 'weather'] = 9
#############wether量化###############

#############处理缺失时间点##########
count = 0
Idx = 0  ####索引记录####
beijing_17_18_meo_list = np.array(beijing_17_18_meo[['longitude','latitude','temperature','pressure','humidity','wind_direction','wind_speed','weather']]).tolist()
for i in tqdm(np.array(beijing_17_18_meo['utc_time']).tolist()):
    if i in train_dict.keys():
        train_dict[i].extend(beijing_17_18_meo_list[Idx])
    if i in test_dict.keys():
        test_dict[i].extend(beijing_17_18_meo_list[Idx])
        
    Idx += 1
######################处理缺失时间点完毕#####################
    
beijing_17_18_meo.head()

100%|██████████| 157471/157471 [00:00<00:00, 965136.11it/s]


,Unnamed: 0,station_id,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed,weather
0,32,shunyi_meo,116.615278,40.126667,2017-02-01 00:00:00,-4.4,1036.6,23.0,52.0,3.6,0
1,33,shunyi_meo,116.615278,40.126667,2017-02-01 01:00:00,-2.9,1037.0,20.0,42.0,3.2,0
2,34,shunyi_meo,116.615278,40.126667,2017-02-01 02:00:00,-1.6,1037.0,17.0,39.0,4.2,0
3,35,shunyi_meo,116.615278,40.126667,2017-02-01 03:00:00,-0.2,1036.4,14.0,52.0,3.8,0
4,36,shunyi_meo,116.615278,40.126667,2017-02-01 04:00:00,0.6,1035.7,13.0,57.0,3.0,0


In [349]:
############meo 缺失填补######################
import copy
train_dict['2018-01-08 09:00:00'] = copy.deepcopy(train_dict['2018-01-08 12:00:00'])
train_dict['2018-01-08 11:00:00'] = copy.deepcopy(train_dict['2018-01-08 12:00:00'])
train_dict['2018-01-08 10:00:00'] = copy.deepcopy(train_dict['2018-01-08 12:00:00'])
train_dict['2017-03-27 06:00:00'] = copy.deepcopy(train_dict['2017-03-27 07:00:00'])
train_dict['2017-08-16 04:00:00'] = copy.deepcopy(train_dict['2017-08-16 06:00:00'])
train_dict['2017-08-16 05:00:00'] = copy.deepcopy(train_dict['2017-08-16 06:00:00'])
train_dict['2017-04-17 03:00:00'] = copy.deepcopy(train_dict['2017-04-17 04:00:00'])
train_dict['2017-03-01 03:00:00'] = copy.deepcopy(train_dict['2017-03-01 04:00:00'])
train_dict['2017-03-07 09:00:00'] = copy.deepcopy(train_dict['2017-03-07 10:00:00'])

In [350]:
############beijing 17-18 网格点 气象数据################
beijing_17_18_meo_grid = pd.read_csv('/home/lifuzhen/KDD2018/Beijing_historical_meo_grid.csv')

count = 0
index =[]

for i in tqdm(beijing_17_18_meo_grid['utc_time']):
    if i.split('-')[1] == '01' and i.split('-')[0] == '2017':
        index.append(count)
    count += 1

beijing_17_18_meo_grid.drop(index , inplace= True)
beijing_17_18_meo_grid.drop_duplicates(inplace=True)        ##删除重行
beijing_17_18_meo_grid.reset_index(drop=True, inplace=True)###重置索引
count = 0
Idx = 0  ####索引记录####

# observe = len(train_dict['2018-01-08 10:00:00'])


beijing_17_18_meo_grid_list = np.array(beijing_17_18_meo_grid[['longitude','latitude','temperature','pressure','humidity','wind_direction','wind_speed/kph']]).tolist()
for i in tqdm(beijing_17_18_meo_grid['utc_time']):
    if i in train_dict.keys():
        train_dict[i].extend(beijing_17_18_meo_grid_list[Idx])
#         if len(train_dict['2018-01-08 10:00:00']) > observe:
#             print(i)
#             observe = len(train_dict['2018-01-08 10:00:00'])
        
    if i in test_dict.keys():
        test_dict[i].extend(beijing_17_18_meo_grid_list[Idx])
        
    Idx += 1
print(count)
beijing_17_18_meo_grid.head()

100%|██████████| 6550362/6550362 [00:04<00:00, 1386654.27it/s]

0


,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph
0,beijing_grid_000,115.0,39.0,2017-02-01 00:00:00,-9.05,997.13,31.10,101.31,1.10
1,beijing_grid_001,115.0,39.1,2017-02-01 00:00:00,-9.19,991.51,30.66,92.05,0.58
2,beijing_grid_002,115.0,39.2,2017-02-01 00:00:00,-9.62,974.64,29.33,301.15,1.09
3,beijing_grid_003,115.0,39.3,2017-02-01 00:00:00,-10.05,957.77,28.00,295.20,2.71
4,beijing_grid_004,115.0,39.4,2017-02-01 00:00:00,-10.23,938.74,26.71,310.09,5.48


In [351]:
#####字典变表格  方便读数据############
train_frame = pd.DataFrame.from_dict(train_dict).T
test_frame = pd.DataFrame.from_dict(test_dict).T


In [352]:
##########提取aq 48小时滑窗##########
win = 48

for i in tqdm(range(280)):  ##训练数据
    count = 0
    for j in train_frame[i]:
        if count >= win:
            train_dict[train_frame.index[count]].extend(np.array(train_frame[i][count-win:count]).tolist())
        count += 1
        
for i in tqdm(range(280)):  ##测试数据
    count = 0
    for j in test_frame[i]:
        if count >= win:
            test_dict[test_frame.index[count]].extend(np.array(test_frame[i][count-win:count]).tolist())
        count += 1

100%|██████████| 280/280 [00:15<00:00, 18.26it/s]


In [353]:
##########标签值 PM10 PM2.5 O3##########

for i in tqdm(range(280)):  ##训练数据
    count = 0
    for j in train_frame[i]:
        if count+48 < len(train_frame[i]) and (i%8 == 0 or i%8 == 1 or i%8 == 4):
            train_dict[train_frame.index[count]].append(np.array(train_frame[i][count+48]).tolist())
        count += 1
        
for i in tqdm(range(280)):  ##测试数据
    count = 0
    for j in test_frame[i]:
        if count+48 < len(test_frame[i]) and (i%8 == 0 or i%8 == 1 or i%8 == 4):
            test_dict[test_frame.index[count]].append(np.array(test_frame[i][count+48]).tolist())
        count += 1

100%|██████████| 280/280 [00:01<00:00, 164.57it/s]


In [359]:
pop_list = []
for i in train_dict.keys():
    if i.split(' ')[0] == '2017-02-01' or i.split(' ')[0] == '2017-02-02' or i.split(' ')[0] == '2017-05-16' or i.split(' ')[0] == '2017-05-17' or i.split(' ')[0] == '2017-05-18' or i.split(' ')[0] == '2017-07-81' or i.split(' ')[0] == '2017-07-09' or i.split(' ')[0] == '2017-07-10' or i.split(' ')[0] == '2018-01-29' or i.split(' ')[0] == '2018-01-30' or i.split(' ')[0] == '2018-01-31':
        pop_list.append(i)
for i in pop_list:
    train_dict.pop(i)

pop_list = []
for i in test_dict.keys():
    if i.split(' ')[0] == '2017-06-01' or i.split(' ')[0] == '2017-06-02' or i.split(' ')[0] == '2017-06-29' or i.split(' ')[0] == '2017-06-30' or i.split(' ')[0] == '2017-07-01':
        pop_list.append(i)
for i in pop_list:
    test_dict.pop(i)